# 02 Feature Engineering & Enriched Datasets

This notebook generates the final analytics-ready datasets:
1. `ipl_ball_enriched.csv` (Ball Level Fact)
2. `ipl_player_metrics.csv` (Player Level Dimension)
3. `ipl_team_metrics.csv` (Team Level Dimension)

**Logic includes:**
- Merging Deliveries and Matches.
- Calculating **Consistency Index** for players.
- Calculating **Match Intensity Score** (Fan Engagement).
- Enriching with external metadata (`Players.xlsx`, `teamwise_home_and_away.csv`).

In [ ]:
import pandas as pd
import numpy as np
import os

PROCESSED_DIR = '../data/processed/'
RAW_DIR = '../data/raw/'

# --- 1. Load Clean Core Data ---
print("Loading clean core data...")
matches = pd.read_csv(f'{PROCESSED_DIR}matches_clean.csv')
deliveries = pd.read_csv(f'{PROCESSED_DIR}deliveries_clean.csv')

# Merge to get match details on every ball (Left Join on match_id)
# deliveries.csv should have match_id if cleaned correctly, or id. 
# Our 01 notebook didn't explicitly rename match_id in deliveries! 
# Let's check schema assumption. Usually deliveries has match_id.
if 'match_id' in deliveries.columns:
    ipl_ball = deliveries.merge(matches, on='match_id', how='left')
else:
    # Fallback if column name differs
    print("Renaming match_id in deliveries...")
    ipl_ball = deliveries.rename(columns={'id': 'match_id'}).merge(matches, on='match_id', how='left')

print(f"Ball Enriched Shape: {ipl_ball.shape}")
ipl_ball.to_csv(f'{PROCESSED_DIR}ipl_ball_enriched.csv', index=False)

# --- 2. Player Level Metrics ---
print("Calculating Player Metrics...")

# Filter for Batting Stats
# Balls faced: excludes wides
ipl_ball['is_legal_delivery'] = (ipl_ball['wide_runs'] == 0).astype(int) if 'wide_runs' in ipl_ball.columns else 1

# Group by Batsman + Season
player_groups = ipl_ball.groupby(['batsman', 'season'])
player_stats = player_groups.agg({
    'match_id': 'nunique', 
    'batsman_runs': 'sum', 
    'is_legal_delivery': 'sum', 
    'is_wicket': 'sum'
}).rename(columns={
    'match_id': 'matches_played',
    'batsman_runs': 'total_runs',
    'is_legal_delivery': 'balls_faced',
    'is_wicket': 'dismissals'
})

# Derived Metrics
player_stats['batting_average'] = np.where(player_stats['dismissals'] > 0, 
                                            player_stats['total_runs'] / player_stats['dismissals'], 
                                            player_stats['total_runs'])
player_stats['strike_rate'] = np.where(player_stats['balls_faced'] > 0, 
                                        (player_stats['total_runs'] / player_stats['balls_faced']) * 100, 
                                        0)
player_stats['avg_runs_per_match'] = player_stats['total_runs'] / player_stats['matches_played']

# Consistency Index (Std Dev)
run_std = ipl_ball.groupby(['batsman', 'season', 'match_id'])['batsman_runs'].sum().groupby(['batsman', 'season']).std()
player_stats['std_dev_runs'] = run_std
player_stats['consistency_index'] = np.where(player_stats['std_dev_runs'] > 0, 
                                              player_stats['avg_runs_per_match'] / player_stats['std_dev_runs'], 
                                              0)

# Enrich with Players.xlsx
try:
    players_meta = pd.read_excel(f'{RAW_DIR}Players.xlsx')
    # Left join on Player_Name
    player_stats = player_stats.reset_index()
    player_stats = pd.merge(player_stats, players_meta, left_on='batsman', right_on='Player_Name', how='left')
except Exception as e:
    print(f"Could not load Players.xlsx: {e}")

print(f"Player Metrics Shape: {player_stats.shape}")
player_stats.to_csv(f'{PROCESSED_DIR}ipl_player_metrics.csv', index=False)

# --- 3. Team Level Metrics & Fan Engagement ---
print("Calculating Team Metrics...")

# Match Intensity Score
# Formula: Total Runs + (Wickets * 20) + (Boundaries * 5)
ipl_ball['is_boundary'] = ipl_ball['batsman_runs'].isin([4, 6]).astype(int)

match_agg = ipl_ball.groupby(['match_id', 'season', 'winner', 'venue', 'team1', 'team2']).agg({
    'total_runs': 'sum',
    'is_wicket': 'sum',
    'is_boundary': 'sum'
}).reset_index()

match_agg['match_intensity'] = (match_agg['total_runs']) + \
                                 (match_agg['is_wicket'] * 20) + \
                                 (match_agg['is_boundary'] * 5)

# Restructure to Team-Season Grain
t1 = match_agg[['season', 'team1', 'match_id', 'winner', 'match_intensity']].rename(columns={'team1': 'team'})
t2 = match_agg[['season', 'team2', 'match_id', 'winner', 'match_intensity']].rename(columns={'team2': 'team'})
team_match = pd.concat([t1, t2], ignore_index=True)

team_match['is_win'] = (team_match['team'] == team_match['winner']).astype(int)

team_metrics = team_match.groupby(['team', 'season']).agg({
    'match_id': 'nunique',
    'is_win': 'sum',
    'match_intensity': 'mean'
}).rename(columns={
    'match_id': 'matches_played',
    'is_win': 'matches_won',
    'match_intensity': 'avg_match_intensity'
}).reset_index()

team_metrics['win_rate'] = (team_metrics['matches_won'] / team_metrics['matches_played']) * 100

# Enrich with Home/Away
try:
    home_away = pd.read_csv(f'{RAW_DIR}teamwise_home_and_away.csv')
    # Standardize names if needed (assuming simple join based on our previous debug)
    # Check if season exists
    join_cols = ['team', 'season'] if 'season' in home_away.columns else ['team']
    team_metrics = pd.merge(team_metrics, home_away, on=join_cols, how='left')
except Exception as e:
    print(f"Could not load teamwise_home_and_away.csv: {e}")

print(f"Team Metrics Shape: {team_metrics.shape}")
team_metrics.to_csv(f'{PROCESSED_DIR}ipl_team_metrics.csv', index=False)

print("✅ All Datasets Generated Successfully.")